In [ ]:
import pandas as pd
import os


file_pattern = 'ppg_g_part{}.csv'
num_files = 7
file_list = [file_pattern.format(i) for i in range(1, num_files + 1)]

# 데이터를 저장할 빈 리스트 초기화
all_data = []

print("CSV 파일 읽기 시작...")

# 1. 파일 반복 및 읽기
for filename in file_list:
    if os.path.exists(filename):
        df = pd.read_csv(filename, header=None)
        all_data.append(df)
        print(f"성공적으로 {filename} 파일의 {len(df)} 행을 읽었습니다.")
    else:
        print(f"경고: {filename} 파일이 존재하지 않아 건너뜁니다.")

print("파일 합치기 시작...")

# 2. 데이터프레임 수직 합치기 (axis=0)
if all_data:
    combined_df = pd.concat(all_data, axis=0, ignore_index=True)
    
    total_rows = len(combined_df)
    total_cols = combined_df.shape[1]
    
    print("✅ 파일 합치기 완료!")
    print(f"최종 데이터프레임 크기: {total_rows} 행 x {total_cols} 열")

else:
    print("❌ 합칠 수 있는 파일이 하나도 없습니다.")

CSV 파일 읽기 시작...
성공적으로 ppg_g_part1.csv 파일의 1000000 행을 읽었습니다.
성공적으로 ppg_g_part2.csv 파일의 1000000 행을 읽었습니다.
성공적으로 ppg_g_part3.csv 파일의 1000000 행을 읽었습니다.
성공적으로 ppg_g_part4.csv 파일의 1000000 행을 읽었습니다.
성공적으로 ppg_g_part5.csv 파일의 1000000 행을 읽었습니다.
성공적으로 ppg_g_part6.csv 파일의 1000000 행을 읽었습니다.
성공적으로 ppg_g_part7.csv 파일의 405120 행을 읽었습니다.
파일 합치기 시작...
✅ 파일 합치기 완료!
최종 데이터프레임 크기: 6405120 행 x 1 열


In [ ]:
import numpy as np

temp_df = pd.read_csv('wrist_temperature.csv', header=None)
ppg_per_temp = 128 * 2  # 256 (128Hz * 2초)

# 2. PPG 데이터의 총 행 수만큼의 길이로 빈 배열 생성
num_ppg_rows = len(combined_df)
temp_values_expanded = np.empty(num_ppg_rows)
temp_values_expanded[:] = np.nan 

print(f"PPG 데이터 총 행 수: {num_ppg_rows}행")

# 3. 온도 측정값을 256 간격마다 배치
# temp_df는 단일 열(0번 열)을 가집니다.
for i in range(len(temp_df)):
    # i * 256: 온도 값이 시작되어야 할 PPG 인덱스 위치
    start_index = i * ppg_per_temp
    
    # PPG 배열의 인덱스 위치가 범위를 벗어나지 않도록 확인 (더 짧은 쪽에 맞춤)
    if start_index < num_ppg_rows:
        temperature_value = temp_df.iloc[i, 0] # i행, 0번 열의 값
        
        # 해당 PPG 인덱스 위치에 온도 값을 삽입
        temp_values_expanded[start_index] = temperature_value
        print(f"온도 {i+1}번째 값 ({temperature_value:.2f}°C)을 PPG 인덱스 {start_index}에 배치")
    else:
        # PPG 데이터가 온도 배치 지점보다 먼저 끝나면, 루프 종료
        print(f"PPG 데이터({num_ppg_rows}행)가 {start_index}보다 짧아 온도 배치 루프 종료.")
        break
        
print("---")

# 4. 확장된 온도 데이터를 데이터프레임으로 변환
temp_expanded_df = pd.DataFrame(
    temp_values_expanded, 
    index=combined_df.index,
    columns=['wrist_temperature']
)

# 5. 선형 보간 수행
temp_interpolated = temp_expanded_df.interpolate(method='linear')

# 6. 두 df 병합
combined_final_df = combined_df.join(temp_interpolated, how='left')

print("✅ 온도 데이터 배치, 선형 보간 및 병합 완료!")
print(f"최종 병합 데이터프레임 크기: {len(combined_final_df)} 행 x {combined_final_df.shape[1]} 열")
print("첫 5행:")
print(combined_final_df.head())

PPG 데이터 총 행 수: 6405120행
온도 1번째 값 (30.34°C)을 PPG 인덱스 0에 배치
온도 2번째 값 (30.36°C)을 PPG 인덱스 256에 배치
온도 3번째 값 (30.37°C)을 PPG 인덱스 512에 배치
온도 4번째 값 (30.38°C)을 PPG 인덱스 768에 배치
온도 5번째 값 (30.39°C)을 PPG 인덱스 1024에 배치
온도 6번째 값 (30.40°C)을 PPG 인덱스 1280에 배치
온도 7번째 값 (30.41°C)을 PPG 인덱스 1536에 배치
온도 8번째 값 (30.42°C)을 PPG 인덱스 1792에 배치
온도 9번째 값 (30.43°C)을 PPG 인덱스 2048에 배치
온도 10번째 값 (30.43°C)을 PPG 인덱스 2304에 배치
온도 11번째 값 (30.44°C)을 PPG 인덱스 2560에 배치
온도 12번째 값 (30.45°C)을 PPG 인덱스 2816에 배치
온도 13번째 값 (30.46°C)을 PPG 인덱스 3072에 배치
온도 14번째 값 (30.47°C)을 PPG 인덱스 3328에 배치
온도 15번째 값 (30.48°C)을 PPG 인덱스 3584에 배치
온도 16번째 값 (30.49°C)을 PPG 인덱스 3840에 배치
온도 17번째 값 (30.49°C)을 PPG 인덱스 4096에 배치
온도 18번째 값 (30.50°C)을 PPG 인덱스 4352에 배치
온도 19번째 값 (30.51°C)을 PPG 인덱스 4608에 배치
온도 20번째 값 (30.51°C)을 PPG 인덱스 4864에 배치
온도 21번째 값 (30.52°C)을 PPG 인덱스 5120에 배치
온도 22번째 값 (30.53°C)을 PPG 인덱스 5376에 배치
온도 23번째 값 (30.53°C)을 PPG 인덱스 5632에 배치
온도 24번째 값 (30.54°C)을 PPG 인덱스 5888에 배치
온도 25번째 값 (30.55°C)을 PPG 인덱스 6144에 배치
온도 26번째 값 (30.56°C)을 PPG 인덱스 6400에 배치
온도 

In [59]:
combined_final_df.head()

,0,wrist_temperature
0,0.477945,30.343066
1,0.478254,30.343113
2,0.477734,30.343160
3,0.478363,30.343207
4,0.478128,30.343253


In [60]:
# 현재 combined_final_df의 총 행 수 (N)
N = len(combined_final_df)

# 샘플링 빈도 (Hz)
fs = 128

# 총 데이터 시간 (초)
total_seconds = N / fs

# 시간(시, 분, 초) 계산
hours = int(total_seconds // 3600)
minutes = int((total_seconds % 3600) // 60)
seconds = total_seconds % 60

# 소수점 둘째 자리까지 표현하기 위해 포맷팅
formatted_seconds = f"{seconds:.2f}"


print(f"총 데이터 행 수: {N} 행")
print(f"총 데이터 시간 (초): {total_seconds} 초")
print(f"✅ 최종 데이터 길이: {hours} 시간 {minutes} 분 {formatted_seconds} 초")

총 데이터 행 수: 6405120 행
총 데이터 시간 (초): 50040.0 초
✅ 최종 데이터 길이: 13 시간 54 분 0.00 초


In [ ]:
final_output_df = combined_final_df[[combined_final_df.columns[0], 'wrist_temperature']].copy()

final_output_df.columns = ['ppg_g', 'temperature']

output_filename = 'user_16.csv'

final_output_df.to_csv(
    output_filename, 
    index=True, 
    header=True,
    index_label=''
)

print("---")
print(f"✅ 데이터 저장 완료!")
print(f"파일 이름: **{output_filename}**")
print("저장 형식: **Index, ppg_g, temperature** 헤더 및 인덱스 모두 포함")
print("---")

---
✅ 데이터 저장 완료!
파일 이름: **user_16.csv**
저장 형식: **Index, ppg_g, temperature** 헤더 및 인덱스 모두 포함
---
